In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
alldata_new = pickle.load(open('output/20topics_newpreprocess_dict.p', 'rb'))

In [3]:
term_topic = alldata_new['term_topic']
doc_topic = alldata_new['doc_topic']
doc_length = alldata_new['doc_length']
term_frequency = alldata_new['term_frequency']
terms = alldata_new['terms']

In [4]:
# structure of term_topic
year = 0
topic = 0
term = 0
prob = term_topic[year,topic,term]
prob

0.00011342876027732682

In [5]:
topics = range(term_topic.shape[1])

In [6]:
# topic label = most frequent word given topic

def topic_label(topic, term_topic, terms):
    term_freqs = np.sum(term_topic[:,topic,:], axis = 0)
    max_term = np.argsort(-term_freqs)[0]
    return(terms[max_term])

topic_labels = [topic_label(topic, term_topic, terms) for topic in topics]

topic_labels

['spatial',
 'face',
 'speaker',
 'action',
 'trial',
 'relational',
 'reasoning',
 'probability',
 'stimulus',
 'concept',
 'speech',
 'child',
 'category',
 'rule',
 'sentence',
 'movement',
 'causal',
 'network',
 'student',
 'decision']

In [7]:

### GOAL:  p(topic|year)

# create year_doc, that shows which document belongs to which year

# use doc_topic, where doc_topic[doc,topic] has p(topic|doc)

# create year_doc_topic.csv, 
# which has the probability of a given topic in a given document in a given year


In [8]:
## year_doc:
## which document belongs to which year

# copy pasted from alex
year_doc_count= [167,
 196,
 156,
 207,
 249,
 404,
 378,
 271,
 201,
 768,
 673,
 823,
 431,
 681,
 498,
 597,
 464,
 657]

x = []
year = 0
doc_id = 0
while year < len(year_doc_count):
    if year_doc_count[year] == 0:
        year += 1
    else:
        year_doc_count[year] -= 1
        x.append({'year': year, 'doc_id': doc_id})
        doc_id += 1
year_doc = pd.DataFrame(x)
year_doc = year_doc[['year','doc_id']]
year_doc.head()

,year,doc_id
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


In [9]:
## create year_doc_topic

# replicate year_doc for each topic
x = []
for topic in topics:
    dd = year_doc.copy()
    dd['topic'] = topic
    dd['topic_label'] = topic_labels[topic]
    x.append(dd)
year_doc_topic = pd.concat(x)


In [10]:
# use doc_topic, where doc_topic[doc,topic] has p(topic|doc)
doc_topic[0,0]

0.0010869565217391309

In [11]:
# given year and topic, go through docs from that year and get p(topic|doc)
def get_prob(row):
    return(doc_topic[row.doc_id, row.topic])

year_doc_topic['prob'] = year_doc_topic.apply(get_prob, axis=1)  # apply get_prob for each row (axis=1)

In [12]:
# save csv
year_doc_topic.to_csv("year_doc_topic.csv", index=False)